In [1]:
test_input = """O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#...."""

In [2]:
input = open("inputs/14").read()

In [3]:
import numpy as np

def parse_board(input):
    return np.array(list(map(list, input.splitlines())))

In [4]:
board = parse_board(test_input)
# board = parse_board(input)
board

array([['O', '.', '.', '.', '.', '#', '.', '.', '.', '.'],
       ['O', '.', 'O', 'O', '#', '.', '.', '.', '.', '#'],
       ['.', '.', '.', '.', '.', '#', '#', '.', '.', '.'],
       ['O', 'O', '.', '#', 'O', '.', '.', '.', '.', 'O'],
       ['.', 'O', '.', '.', '.', '.', '.', 'O', '#', '.'],
       ['O', '.', '#', '.', '.', 'O', '.', '#', '.', '#'],
       ['.', '.', 'O', '.', '.', '#', 'O', '.', '.', 'O'],
       ['.', '.', '.', '.', '.', '.', '.', 'O', '.', '.'],
       ['#', '.', '.', '.', '.', '#', '#', '#', '.', '.'],
       ['#', 'O', 'O', '.', '.', '#', '.', '.', '.', '.']], dtype='<U1')

In [5]:
import math

def split_list(lst, c):
    l = len(lst)

    idxs = [i for i, v in enumerate(lst) if v == c]
    return [lst[i+1:j] for i, j in zip([-1]+idxs, idxs + [l])]

In [6]:
def split_list(lst):
    lsts = []
    l = []

    for i, v in enumerate(lst):
        if v == '#':
            lsts.append(l)
            lsts.append([v])
            l = []
        else:
            l.append(v)

            if i == len(lst) - 1:
                lsts.append(l)
    
    return [np.array(l) for l in lsts if len(l) > 0] 

In [7]:
# [0:idx1], [idx1+1:idx2], [idx2+1:idx3], ..., [idxn+1:]

In [8]:
test_col = board[:, 9]
test_col

array(['.', '#', '.', 'O', '.', '#', 'O', '.', '.', '.'], dtype='<U1')

In [9]:
def roll_north(board):
    new_board = board.copy()

    for i in range(board.shape[1]):
        col = board[:, i]

        new_splits = []
        for l in split_list(col):
            num_os = (l == 'O').sum()

            if num_os > 0:
                a = np.full(num_os, 'O')
                b = np.full(len(l) - num_os, '.')

                new_array = np.concatenate([a, b])
                new_splits.append(new_array)
            else:
                new_splits.append(l)

        combined = np.concatenate(new_splits)

        new_board[:, i] = combined
    
    return new_board

In [10]:
def count_load(board):
    num_rows = board.shape[0]

    sm = 0
    for row in range(num_rows):
        row_mult = num_rows - row
        num_os_in_row = (board[row] == 'O').sum()

        sm += row_mult * num_os_in_row
    return sm

In [11]:
count_load(roll_north(board))

136

In [12]:
def right_rotate(a):
    return np.rot90(a, k=1, axes=(1, 0))

In [13]:
def roll_north_and_rotate(board):
    return right_rotate(roll_north(board))

def roll_cycle(board):
    return roll_north_and_rotate(roll_north_and_rotate(roll_north_and_rotate(roll_north_and_rotate(board))))

In [14]:
orientations = ['N', 'W', 'S', 'E']

In [15]:
roll_num_to_board = dict()
board_id_to_roll_num = dict()

In [16]:
def get_board_id(board):
    return tuple(board.flatten())

In [17]:
CYCLE_NUM = 1000000000

In [18]:
import itertools

# board = parse_board(test_input)
board = parse_board(input)

first = None
next = None

for i in itertools.count(1):
    board = roll_cycle(board)
    roll_num_to_board[i] = board
    board_id = get_board_id(board)

    if board_id in board_id_to_roll_num:
        first = board_id_to_roll_num[board_id]
        next = i
        print(f"Found cycle at roll {i} and roll {board_id_to_roll_num[board_id]}")
        break

    board_id_to_roll_num[board_id] = i

Found cycle at roll 199 and roll 171


In [19]:
period = next - first
first, next, period

(171, 199, 28)

In [21]:
count_load(roll_num_to_board[((CYCLE_NUM - first) % period) + first])

104619